# Print Output Table From Bolocalc

This notebook, once pointed to an experiment directory, prints out a variety of relevant parameters (optical loading, NEPs, NETs) for each channel.  The current purpose of this is to compare with the outputs of Barkats' code, so the tables are formatted with that in mind.

In [30]:
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
%matplotlib inline
plt.rcParams.update({'font.size': 18})
plt.rcParams['figure.figsize'] = [14, 8]

In [31]:
# Import unpack.py from the BoloCalc/src/ directory
BoloCalcPath = 'BoloCalc/'
src_path = BoloCalcPath+'src'
if src_path not in sys.path:
    sys.path.append(src_path)

import unpack as up
# Instantiate an Unpack object
unpack = up.Unpack()    

In [32]:
exp_dir = BoloCalcPath+'Experiments/JR3/Tophat_bands'

In [33]:
unpack.unpack_sensitivities(exp_dir)

The next command shows that all the input Psat vectors are in any one of the camera files.  ie, MF and UHF input psat vectors are in the LF dictionary.

In [34]:
unpack.sens_outputs['Tophat_bands']['SAT']['LF']['Summary']['LF_1']

{'Num Det': [1.0, 0, 0],
 'Optical Throughput': [0.302, 0.0, 0.0],
 'Optical Power': [0.54, 0.0, 0.0],
 'Telescope Temp': [9.78, 0.0, 0.0],
 'Sky Temp': [8.2, 0.0, 0.0],
 'Photon NEP': [10.13, 0.0, 0.0],
 'Bolometer NEP': [9.23, 0.0, 0.0],
 'Readout NEP': [0.0, 0.0, 0.0],
 'Detector NEP': [13.7, 0.0, 0.0],
 'Detector NET_CMB': [330.3, 0.0, 0.0],
 'Detector NET_RJ': [322.6, 0.0, 0.0],
 'Array NET_CMB': [369.24, 0.0, 0.0],
 'Array NET_RJ': [360.69, 0.0, 0.0],
 'Correlation Factor': [1.0, 0.0, 0.0],
 'CMB Map Depth': [231.23, 0.0, 0.0],
 'RJ Map Depth': [225.88, 0.0, 0.0]}

We want to print, in this order

- Total optical power
- NEP_photon
- NEP_phonon, NEP_phonon modified for barkats Flink
- End to End optical efficiency
- NET_CMB, NET_CMB modified for Barkats Flink

In [42]:
#F_barkats_factor = 1.187 # multiply phonon NEP by this to get Barkat's value, for 0.25 and 0.5K ends, n=3
tubes = ['LF','MF','UHF']
chs = ['_1','_2']
print('Center freqs = 30,40,95,150,220,280 GHz')
print('Fractional bandwidth = 0.24')
print('beta = 2, n = 3')
print('Run bolocalc manually with Barkats Flink = 0.667')
for tube in tubes:
    #print(tube)
    for ch in chs:
        chname = tube+ch
        print(' '+chname)
        data = unpack.sens_outputs['Tophat_bands']['SAT'][tube]['Summary'][chname]
        
        NEP_photon = data['Photon NEP'][0]
        NEP_phonon = data['Bolometer NEP'][0]
        NEP_total = np.sqrt(NEP_photon**2+NEP_phonon**2)
        NET_CMB = data['Detector NET_CMB'][0]
        dPdTcmb = NEP_total/(NET_CMB*np.sqrt(2))
        #
        
        print('   P_optical = {0:3.3f} pW'.format(data['Optical Power'][0] ))
        print('   NEP_photon = {0:3.3f} aWrtHz'.format(NEP_photon))
        print('   NEP_phonon = {0:3.3f} aWrtHz'.format(NEP_phonon))
        print('   NEP_total = {0:3.3f} aWrtHz'.format(NEP_total))
        print('   Optical efficiency = {0:3.3f}'.format(data['Optical Throughput'][0]))
        print('   NET_CMB = {0:3.1f} uKrtsec'.format(NET_CMB))
        print('   dPdTcmb = {0:4.4f} pW/uK '.format(dPdTcmb))
    print('')

Center freqs = 30,40,95,150,220,280 GHz
Fractional bandwidth = 0.24
beta = 2, n = 3
Run bolocalc manually with Barkats Flink = 0.667
 LF_1
   P_optical = 0.540 pW
   NEP_photon = 10.130 aWrtHz
   NEP_phonon = 9.230 aWrtHz
   NEP_total = 13.704 aWrtHz
   Optical efficiency = 0.302
   NET_CMB = 330.3 uKrtsec
   dPdTcmb = 0.0293 pW/uK 
 LF_2
   P_optical = 0.950 pW
   NEP_photon = 15.560 aWrtHz
   NEP_phonon = 12.250 aWrtHz
   NEP_total = 19.803 aWrtHz
   Optical efficiency = 0.294
   NET_CMB = 375.0 uKrtsec
   dPdTcmb = 0.0373 pW/uK 

 MF_1
   P_optical = 2.010 pW
   NEP_photon = 24.700 aWrtHz
   NEP_phonon = 17.830 aWrtHz
   NEP_total = 30.463 aWrtHz
   Optical efficiency = 0.294
   NET_CMB = 292.1 uKrtsec
   dPdTcmb = 0.0737 pW/uK 
 MF_2
   P_optical = 2.640 pW
   NEP_photon = 30.180 aWrtHz
   NEP_phonon = 20.400 aWrtHz
   NEP_total = 36.428 aWrtHz
   Optical efficiency = 0.299
   NET_CMB = 301.0 uKrtsec
   dPdTcmb = 0.0856 pW/uK 

 UHF_1
   P_optical = 4.650 pW
   NEP_photon = 46.740 

# Barkat's calculator results, for comparison
Run Barkat's web calculator with the same elements leading to the same overall optical efficiency.  Tune the window temperature to make the P_opticals the same.  This will make the photon noise, the G, the phonon noise, and responsivity all the same (or about the same).  Then, compare the NETs.

(Runs by JR, on 12/8/2020, at http://bicep.rc.fas.harvard.edu/dbarkats/postings/NET_calculator/index.html
with custom settings.)

In [50]:
barkats_values = {}

barkats_values['LF_1']={}
barkats_values['LF_1']['T_window']= 297.
barkats_values['LF_1']['NET_cmb']= 327.4
barkats_values['LF_1']['dPdT_cmb']= 0.0297 #pW/K
barkats_values['LF_1']['P_opt']= 0.540     #pW
#
barkats_values['LF_2']={}
barkats_values['LF_2']['T_window']= 264.
barkats_values['LF_2']['NET_cmb']= 375.11
barkats_values['LF_2']['dPdT_cmb']= 0.0377 #pW/K
barkats_values['LF_2']['P_opt']= 0.950     #pW
#
barkats_values['MF_1']={}
barkats_values['MF_1']['T_window']= 256.8
barkats_values['MF_1']['NET_cmb']= 295.39
barkats_values['MF_1']['dPdT_cmb']=  0.0740 #pW/K
barkats_values['MF_1']['P_opt']= 2.010     #pW
#
barkats_values['MF_2']={}
barkats_values['MF_2']['T_window']= 256.5
barkats_values['MF_2']['NET_cmb']= 306.96
barkats_values['MF_2']['dPdT_cmb']=   0.0854 #pW/K
barkats_values['MF_2']['P_opt']= 2.640     #pW
#
barkats_values['UHF_1']={}
barkats_values['UHF_1']['T_window']= 206.3
barkats_values['UHF_1']['NET_cmb']= 559.14
barkats_values['UHF_1']['dPdT_cmb']= 0.0694   #pW/K
barkats_values['UHF_1']['P_opt']= 4.650     #pW
#
barkats_values['UHF_2']={}
barkats_values['UHF_2']['T_window']= 180 
barkats_values['UHF_2']['NET_cmb']= 1116.85
barkats_values['UHF_2']['dPdT_cmb']= 0.0477   #pW/K
barkats_values['UHF_2']['P_opt']=  7.330    #pW
#



In [70]:
tubes = ['LF','MF','UHF']
chs = ['_1','_2']
print('Center freqs = 30,40,95,150,220,280 GHz')
print('Fractional bandwidth = 0.24')
print('beta = 2, n = 3')
print('Run bolocalc manually with Barkats Flink = 0.667')
print('Set Barkats T_window to give same P_optical as Bolocalc.')
print('Other parameters all the same.')
for tube in tubes:
    #print(tube)
    for ch in chs:
        chname = tube+ch
        print(' '+chname+'                   Bolocalc            Barkats')
        data = unpack.sens_outputs['Tophat_bands']['SAT'][tube]['Summary'][chname]
        NEP_photon = data['Photon NEP'][0]
        NEP_phonon = data['Bolometer NEP'][0]
        NEP_total = np.sqrt(NEP_photon**2+NEP_phonon**2)
        NET_CMB = data['Detector NET_CMB'][0]
        dPdTcmb = NEP_total/(NET_CMB*np.sqrt(2))
        #
        print('   P_optical (pW)    :   {0:3.3f}               {1:3.3f}'
              .format(data['Optical Power'][0],barkats_values[chname]['P_opt'] ))
        print('   NET_CMB (uKrtsec) :   {0:3.1f}             {1:3.1f}       , ratio = {2:2.3f}'
              .format(NET_CMB, barkats_values[chname]['NET_cmb'], NET_CMB/barkats_values[chname]['NET_cmb']))
        print('   dPdTcmb (pW/uK)   :   {0:4.4f}              {1:4.4f}'
              .format(dPdTcmb, barkats_values[chname]['dPdT_cmb']))
    print('')

Center freqs = 30,40,95,150,220,280 GHz
Fractional bandwidth = 0.24
beta = 2, n = 3
Run bolocalc manually with Barkats Flink = 0.667
Set Barkats T_window to give same P_optical as Bolocalc.
Other parameters all the same.
 LF_1                   Bolocalc            Barkats
   P_optical (pW)    :   0.540               0.540
   NET_CMB (uKrtsec) :   330.3             327.4       , ratio = 1.009
   dPdTcmb (pW/uK)   :   0.0293              0.0297
 LF_2                   Bolocalc            Barkats
   P_optical (pW)    :   0.950               0.950
   NET_CMB (uKrtsec) :   375.0             375.1       , ratio = 1.000
   dPdTcmb (pW/uK)   :   0.0373              0.0377

 MF_1                   Bolocalc            Barkats
   P_optical (pW)    :   2.010               2.010
   NET_CMB (uKrtsec) :   292.1             295.4       , ratio = 0.989
   dPdTcmb (pW/uK)   :   0.0737              0.0740
 MF_2                   Bolocalc            Barkats
   P_optical (pW)    :   2.640               2.6

In [36]:
unpack.unpack_optical_powers(exp_dir)

In [37]:
unpack.pwr_outputs['Tophat_bands']['SAT']['LF']['Summary']['LF_1'].keys()

dict_keys(['CMB', 'ATM', 'Window', 'IR_blocker1', 'IR_blocker2', 'IR_blocker3', 'Ap_stp', 'Lenses', 'Detector'])

In [38]:
tube = 'LF'
ch = tube+'_1'
print(ch)
P_tot = 0
for element in unpack.pwr_outputs['Tophat_bands']['SAT'][tube]['Summary'][ch].keys():
    value = unpack.pwr_outputs['Tophat_bands']['SAT'][tube]['Summary'][ch][element]['Power to Detector'][0]
    P_tot = P_tot + value
    name = element
    if (len(name)<6):
        name = element + "\t"
    print('  '+name+' \t {0:3.3f}'.format(value))
print('\n  Total = \t {0:3.3f}'.format(P_tot))

LF_1
  CMB	 	 0.062
  ATM	 	 0.184
  Window 	 0.174
  IR_blocker1 	 0.047
  IR_blocker2 	 0.022
  IR_blocker3 	 0.019
  Ap_stp 	 0.023
  Lenses 	 0.009
  Detector 	 0.000

  Total = 	 0.540


In [39]:
tube_vec = ['LF','MF','UHF']
ch_vec = ['_1','_2']

print('Atmospheric power on detector')
for tube in tube_vec:
    for ch in ch_vec:
        chname = tube+ch
        value = unpack.pwr_outputs['Tophat_bands']['SAT'][tube]['Summary'][chname]['ATM']['Power to Detector'][0]
        #print(chname+'\t {0:3.3f} pW'.format(value))
        print(value)

Atmospheric power on detector
0.184
0.488
1.024
1.183
2.66
4.889


In [40]:
tube_vec = ['LF','MF','UHF']
ch_vec = ['_1','_2']

print('Total power on detector')
for tube in tube_vec:
    for ch in ch_vec:
        chname = tube+ch
        P_tot = 0
        for element in unpack.pwr_outputs['Tophat_bands']['SAT'][tube]['Summary'][chname].keys():
            P = unpack.pwr_outputs['Tophat_bands']['SAT'][tube]['Summary'][chname][element]['Power to Detector'][0]
            P_tot = P_tot + P
        print(chname+'  Total = \t {0:3.3f}'.format(P_tot))

Total power on detector
LF_1  Total = 	 0.540
LF_2  Total = 	 0.950
MF_1  Total = 	 2.013
MF_2  Total = 	 2.635
UHF_1  Total = 	 4.649
UHF_2  Total = 	 7.330


In [41]:
tube_vec = ['LF','MF','UHF']
ch_vec = ['_1','_2']

print('Element optical efficiency')
element = 'Detector'
print(element)
for tube in tube_vec:
    for ch in ch_vec:
        chname = tube+ch
        value = unpack.pwr_outputs['Tophat_bands']['SAT'][tube]['Summary'][chname][element]['Efficiency'][0]
        print(value) 


Element optical efficiency
Detector
0.4
0.4
0.4
0.401
0.4
0.4
